In [1]:
from quadmodel.inference.forward_model import forward_model
import os
import numpy as np
import matplotlib.pyplot as plt

### Forward modeling image flux ratios with quadmodel

In the following cell, we define the parameters we want to sample in the forward model, and set the lens data class

In [4]:
output_path = os.getcwd() + '/example_inference_output/'
job_index = 1
n_keep = 4
summary_statistic_tolerance = 1e4
lens_data = 'B1422'
from quadmodel.data.b1422 import B1422
lens_data = B1422()
print(lens_data.m)
print(lens_data.macromodel_type)
realization_priors = {}
realization_priors['PRESET_MODEL'] = 'WDM'
realization_priors['sigma_sub'] = ['UNIFORM', 0.0, 0.1]
realization_priors['LOS_normalization'] = ['FIXED', 1.]
realization_priors['log_mc'] = ['UNIFORM', 4.8, 10.0]
realization_priors['log_m_host'] = ['FIXED', 13.3]

macromodel_priors = {}
macromodel_priors['m4_amplitude_prior'] = [np.random.normal, 0.0, 0.01]
macromodel_priors['gamma_macro_prior'] = [np.random.uniform, 1.8, 2.3]
# FOR A CUSTOM SHEAR PRIOR:
# macromodel_priors['shear_strength_prior'] = [np.random.uniform, 0.05, 0.25]

# the present lenses also have built-in shear priors determined based on what values get accepted after running ABC;
# using a brooader prior, you will waste some time exploring parameter space that will get rejected
shear_min, shear_max = lens_data.kwargs_macromodel['shear_amplitude_min'], lens_data.kwargs_macromodel['shear_amplitude_max']
print(shear_min, shear_max)
macromodel_priors['shear_strength_prior'] = [np.random.uniform, shear_min, shear_max]


[0.88  1.    0.474 0.025]
EPL_FIXED_SHEAR_MULTIPOLE
0.08 0.4


### Run the simulation

In [5]:
forward_model(output_path, job_index, lens_data, n_keep, realization_priors, 
              macromodel_priors, tolerance=summary_statistic_tolerance,
                  verbose=True, test_mode=True, save_realizations=True)

reading output to files: 
/Users/danielgilman/Code/quadmodel/notebooks/example_inference_output/job_1/parameters.txt
/Users/danielgilman/Code/quadmodel/notebooks/example_inference_output/job_1/fluxes.txt
starting with 2 samples accepted, 2 remain
existing magnifications:  [[0.611054 1.       0.381723 0.068976]
 [0.692226 1.       0.404612 0.207799]]
samples remaining:  2
importance weight for sample:  1.0
sample (from realization samples):  [0.08483907 9.70426766]
realization contains 32 halos.
['sigma_sub', 'log_mc']
realization hyper-parameters:  [0.08483907 9.70426766]
['source_size_pc']
source/lens parameters:  55.468561659883754
['a_m_4', 'gamma_macro', 'gamma_ext']
macromodel samples:  [-0.00627629  1.91122617  0.10544879]


KeyboardInterrupt: 

### Examine the output

The simulation output includes files named fluxes.txt, parameters.txt, and simulation_output_1, simulation_output_2, etc.

In [ ]:
f = open(output_path + 'job_'+str(job_index)+'/parameters.txt', 'r')
param_names = f.readlines()[0]
print('PARAMETER NAMES:')
print(param_names)
f.close()

accepeted_parameters = np.loadtxt(output_path + 'job_'+str(job_index)+'/parameters.txt', skiprows=1)
print('ACCEPTED PARAMETERS:')
print(accepeted_parameters)
# the first set of parameters are the ones specified in kwargs_realization (see cell #2), the rest are the source size, 
# macromodel parameters, and the last parameter is the summary statistic

accepeted_mags = np.loadtxt(output_path + 'job_'+str(job_index)+'/fluxes.txt')
print('\nACCEPTED MAGNIFICATIONS:')
print(accepeted_mags)

### Visualize accepeted realizations

The pickeled classes in simulation_output allow you to visualize the accepeted realizations

In [ ]:
import pickle
f = open(output_path + 'job_'+str(job_index)+'/simulation_output_1', 'rb')
simulation_output = pickle.load(f)
x_image, y_image = simulation_output.data.x, simulation_output.data.y
lens_system = simulation_output.lens_system

npix = 150
rminmax = 1.25
_r = np.linspace(-rminmax, rminmax, npix)
xx, yy = np.meshgrid(_r, _r)
shape0 = xx.shape
lensmodel, kwargs_lens = lens_system.get_lensmodel()
lensmodel_macro, kwargs_macro = lens_system.get_lensmodel(include_substructure=False)

kappa = lensmodel.kappa(xx.ravel(), yy.ravel(), kwargs_lens).reshape(shape0)
kappa_macro = lensmodel_macro.kappa(xx.ravel(), yy.ravel(), kwargs_lens).reshape(shape0)

In [ ]:
import matplotlib.pyplot as plt

vminmax = 0.05
kappa_subs_nonlinear = kappa - kappa_macro
extent = [-rminmax, rminmax, -rminmax, rminmax]

fig = plt.figure(1)
fig.set_size_inches(8, 8)
plt.imshow(kappa_subs_nonlinear, vmin=-vminmax, vmax=vminmax, origin='lower', cmap='bwr', 
           extent=extent)
plt.scatter(x_image, y_image, color='k')
plt.show()

source_size_pc = accepeted_parameters[0, 4]
lens_system.plot_images(x_image, y_image, source_size_pc, lensmodel, kwargs_lens)

Using the plot_images routine in lens_system, you can also ray trace through the lens system to see what the images look like (or how they would look with an ridiculously large telescope and no PSF) 